In [1]:
import pandas as pd
import numpy as np
import requests
import cv2 
from pytesseract import pytesseract
from PIL import Image
import copy

C:\Users\roshr\AppData\Local\Programs\Python\Python310\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\roshr\AppData\Local\Programs\Python\Python310\Lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\roshr\AppData\Local\Programs\Python\Python310\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [3]:
df = pd.read_json('subspace.json')
df

,expiry_image,expiring_at,expiring_at_from_model,created_at,whatsub_plan
0,https://cdn.subspace.money/w_subscriptions/Xk5...,2023-05-13,2023-05-13,2022-05-15 04:27:17.955815+00:00,"{'duration': 3, 'duration_type': 'months'}"
1,https://cdn.subspace.money/w_subscriptions/PKV...,2023-03-29,None,2022-06-10 11:28:46.053421+00:00,"{'duration': 1, 'duration_type': 'years'}"
2,https://cdn.subspace.money/w_subscriptions/80E...,2022-12-27,2022-12-27,2022-07-21 16:30:53.316893+00:00,"{'duration': 1, 'duration_type': 'months'}"
3,https://cdn.subspace.money/w_subscriptions/JIX...,2022-10-22,2022-08-22,2022-07-22 15:02:53.685566+00:00,"{'duration': 1, 'duration_type': 'months'}"
4,https://cdn.subspace.money/w_subscriptions/p4r...,2022-07-12,None,2022-06-12 05:16:00.847964+00:00,"{'duration': 1, 'duration_type': 'months'}"
...,...,...,...,...,...
6723,https://cdn.subspace.money/w_subscriptions/OYl...,2023-10-07,2023-10-07,2022-10-07 16:03:36.596915+00:00,"{'duration': 12, 'duration_type': 'months'}"
6724,https://cdn.subspace.money/w_subscriptions/5av...,2022-11-07,None,2022-10-07 16:06:06.037587+00:00,"{'duration': 1, 'duration_type': 'months'}"
6725,https://cdn.subspace.money/w_subscriptions/X4e...,2022-12-31,None,2022-10-07 17:25:59.667282+00:00,"{'duration': 1, 'duration_type': 'months'}"
6726,https://cdn.subspace.money/w_subscriptions/qmT...,2022-11-07,None,2022-06-14 08:22:25.590519+00:00,"{'duration': 1, 'duration_type': 'months'}"


Accessing All Images

In [4]:
for i in df.iloc[:1,0]:
    r = requests.get(i)
    print(i)
    with open('test.jpeg', 'wb') as out:
        out.write(r.content)
    im = Image.open("test.jpeg")
im.show()

https://cdn.subspace.money/w_subscriptions/Xk5bMvR8J7I_H6ywBPgSe.jpeg


Image Processing

In [5]:

image = cv2.imread('test.jpeg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray, (3,3), 0)
thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Morph open to remove noise and invert image
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
invert = 255 - opening
cv2.imwrite('converted.jpg',invert)

True

Text Extraction

In [7]:
data = pytesseract.image_to_string(invert, lang='eng', config='--psm 6')
print(data)

3:53 PM|4.8MB/sO 9 v J oll Wei ll We, GO

<

zomato
Gixe) PLUS

First became member on 21 Jan '20

You are a Pro Plus member

9 ‘sc

Free deliveries on all Up ta 30° extra off on
online orders food deliveries

Expires on 13 May '23

Explore pro dining partners

Pro 15% OFF
Biergarten Hakuna Matata 1131 Bar +
Whitefield JP Nagar Indiranagar

- o>

Pro 20% OFF 4.5% Pro 15% OFF


